# EE 5020 Homework 4: Discrete-event simulation with SimPy and Optimization with SciPy

Name: 

CIN: 

## Overview

In this homework, you will practice applying your mechanistic simulation skills to create models that generate datasets you can analyze and optimize. Like the previous homework, you will answer research questions below with your own Markdown and Python Code cells.

## Rubric for this homework

**Make sure you write down the statistical reasoning and justification for any Python code cells created.**  The Python code is only there to support you computationally for your simulation and optimization analyses. Additionally, make sure to justify any conclusions you make to answer questions with quantitative reasoning.

Grading breakdown:

- 25% organization and flow of simulation analysis (including random variable distributions)
- 25% correct simulation code
- 25% optimization setup (identification of parameters and coding of problem)
- 25% correct optimization result

**[Click here to watch a video about Academic Dishonesty in the Electrical and Computer Engineering Department](https://calstatela.zoom.us/rec/share/xWIpPJA8P5itIqGmXv5zjDcHfFO5Qi8hjURVGhkZ7adpwfxDas_6Kd6KdhcDTlAk.a2p_HGobNOEEwXNZ?startTime=1633466851000)**

**Any cheating or academic dishonesty with this homework will result in an automatic zero grade and referral to the College for discipline, including dismissal from the graduate program.**

Please refer to the [Student Handbook](https://www.calstatela.edu/studentconduct/california-code-regulations-standards-student-conduct) for more information.

## Global imports

Write your imports here so you don't have to write imports below.

In [15]:
import numpy as np
import scipy.stats
import scipy.optimize
import simpy as sp
import pandas as pd
from typing import List, Tuple

## Problem 1

**Write and discuss the steps to answering the following research question:** We would like to design a [network packet](https://en.wikipedia.org/wiki/Network_packet) routing system. To do so, we will need to define the system and run a simulation so that we can generate data that will be used for optimization of design parameters. Our router will be a bit of a simpler system, so do not worry about any specifics you know about ethernet or telecommunications in general.  The goal of our router is that it will route packets from devices connected to it through to the internet.

We'll parameterize our router so that it can accept $\alpha$ packet routing connections at any time. Assume that packets take $\beta$ time to be routed through the router. Let's say that each device connected to the router is only willing to wait up to $\gamma$ time for the packet to be routed to its destination.  

Let's also say that our system will allow for prioritization on three levels, so that we can make sure those high-priority packets like those for voice and video calls can route through before lower-priority packets like standard file downloads.  

To model our system, let's say that we have $n$ devices on our network connected to the router.  Each device generates $X_v$ videocalling packets, $X_s$ standard priority packets, and $X_l$ low priority packets at an $X_i$ interval time between each packet submitted to the router.  So, say that this device has a total of $X_v + X_s + X_l$ packets.  I would recommend that you shuffle this set of packets.  This device would submit the first of these packets (in any order) to the router at time 0.  The device would then wait $X_i$ before submitting the next packet (which again can be any of those in this device's collection), and again until the total number of packets has been submitted to the router.

Higher priority packets will not preempt lower priority packets, but they will be able to jump the queue (`simpy.PriorityResource`).

Based on the distributions for the parameters below, what is the average number of packets that make it through the router to the internet, for $n$ = [3, 5, 10, 15]?  All time-related parameters below are in milliseconds. Make sure to set the random seed.
- $\alpha$ = 6
- $\beta$ = exponentially-modified normal continuous random variable at location 1 and scale 1.5
- $\gamma$ = 3
- $X_v$ = uniform discrete random variable between [2,10]
- $X_s$ = uniform discrete random variable between [2,10]
- $X_l$ = uniform discrete random variable between [2,10]
- $X_i$ = binomial discrete random variable with n at 10 and p at 0.5

**Solution:** The easiest strategy here is to just keep a variable that stores the total number of packets that were successfully routed.  If you wanted to, you could do the full log as well and then count from there, which is the best and most flexible strategy, since often you will need more than just one metric when working on a design problem in your career.

We can use a nested for loop to generate all of the packets across the devices at once:

In [35]:
RANDOM_SEED = 42

def packet_process(env: sp.Environment,
                   router: sp.PriorityResource,
                   list_successful_packets: list,
                   list_unsuccessful_packets: list,
                   packet_name: str,
                   prio: int, beta: float,
                   gamma: float, omega: float,
                   start_delay: float):
    device_wait_time_left = gamma
    router_wait_time_left = omega
    packet_routing_time = np.abs(beta)
    timeout_event = None
    
    yield env.timeout(start_delay)  # handle x_i
    with router.request(priority=prio) as routing_slot:
        wait_start = env.now
        device_timeout = env.timeout(device_wait_time_left)
        timeout_event = yield routing_slot | device_timeout
        if device_timeout in timeout_event:
            list_unsuccessful_packets.append(packet_name)
            return
        
        wait_finish = env.now
        device_wait_time_left -= wait_finish - wait_start
        router_wait_time_left -= wait_finish - wait_start
        device_timeout = env.timeout(device_wait_time_left)
        router_timeout = env.timeout(router_wait_time_left)
        timeout_event = yield env.timeout(packet_routing_time) | device_timeout | router_timeout
        if device_timeout in timeout_event or router_timeout in timeout_event:
            list_unsuccessful_packets.append(packet_name)
            return
    
    list_successful_packets.append(packet_name)
    return


def generate_packets(env: sp.Environment, router: sp.PriorityResource,
                     the_rng: np.random.Generator,
                     num_devices: int, list_success: list,
                     list_failure: list, gamma: float, omega: float):
    priorities = {"v": 0, "s": 1, "l": 2}
    dist_beta = scipy.stats.exponnorm(1, 1.5)
    dist_beta.random_state = the_rng
    dist_x_v = scipy.stats.randint(2, 11)
    dist_x_v.random_state = the_rng
    dist_x_s = scipy.stats.randint(2, 11)
    dist_x_s.random_state = the_rng
    dist_x_l = scipy.stats.randint(2, 11)
    dist_x_l.random_state = the_rng
    dist_x_i = scipy.stats.binom(10, 0.5)
    dist_x_i.random_state = the_rng
    for device_id in range(num_devices):
        x_i = dist_x_i.rvs()  # length of start delays
        x_v = dist_x_v.rvs()  # number of V packets
        x_s = dist_x_s.rvs()  # number of S packets
        x_l = dist_x_l.rvs()  # number of L packets
        for pid in range(x_v):
            beta = dist_beta.rvs()
            packet_name = f"Device {device_id} V_Packet {pid}"
            env.process(packet_process(env, router, list_success, list_failure,
                                       packet_name, priorities["v"], beta,
                                       gamma, omega, pid * x_i))
        for pid in range(x_s):
            beta = dist_beta.rvs()
            packet_name = f"Device {device_id} S_Packet {pid}"
            env.process(packet_process(env, router, list_success, list_failure,
                                       packet_name, priorities["s"], beta,
                                       gamma, omega, pid * x_i))
        for pid in range(x_l):
            beta = dist_beta.rvs()
            packet_name = f"Device {device_id} L_Packet {pid}"
            env.process(packet_process(env, router, list_success, list_failure,
                                       packet_name, priorities["l"], beta,
                                       gamma, omega, pid * x_i))


def run_simulation(omega: float, num_devices: int,
                   alpha: int = 6, gamma: float = 3.) -> Tuple[int, int]:
    list_success = []
    list_failure = []
    the_rng = np.random.default_rng(RANDOM_SEED)
    the_env = sp.Environment()
    the_router = sp.PriorityResource(the_env, capacity=alpha)
    generate_packets(the_env, the_router, the_rng, num_devices,
                     list_success, list_failure, gamma, omega)
    the_env.run()
    return len(list_success), len(list_failure)

In [36]:
for n in [3, 5, 10, 15]:
    packets_success, packets_failed = run_simulation(1000, n)
    print(f"{n} devices produced {packets_success} routed, {packets_failed} dropped"
          f", for {packets_success / (packets_failed + packets_success):.2f} success rate")

3 devices produced 41 routed, 13 dropped, for 0.76 success rate
5 devices produced 57 routed, 33 dropped, for 0.63 success rate
10 devices produced 59 routed, 114 dropped, for 0.34 success rate
15 devices produced 66 routed, 185 dropped, for 0.26 success rate


## Problem 2

**Write and discuss the steps to answering the following research question:** Create a function that will automatically run the simulation above, introducing a new constraint named $\omega$, which is the length of time the router is willing to dedicate to routing a packet.  Thus, you will need to modify Problem 1 so that $\omega$ is incorporated as another wait time that will interrupt the packet routing.

Then, we would like to see how $\omega$ affects the number of packets that successfully route through. Make sure that the function sets the same random seed inside the function, so that your simulation parameters are consistent across runs. 

Using `scipy.optimize.minimize_scalar()` ([see reference](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize_scalar.html#scipy.optimize.minimize_scalar)), what is the value of $\omega$ (router processing time allocated) that results in the highest number of packets that successfully route through our router design, where $2 \le \omega \le 10$?

**Solution:** It looks like we can just assume 3 devices is best since it has the highest ratio of successful packets.  So, just define the objective function as the negative of the number of successes to maximize.

In [37]:
def objective(omega: float):
    n_success, n_failure = run_simulation(omega, 3)
    return -n_success


opt_soln = scipy.optimize.minimize_scalar(objective, bounds=(2., 10.))
opt_x = opt_soln.x

for omega in [2., 4., opt_x, 10., 12.]:
    packets_success, packets_failed = run_simulation(omega, 3)
    print(f"{omega} omega: {packets_success} routed, {packets_failed} dropped"
          f", for {packets_success / (packets_failed + packets_success):.2f} success rate")

2.0 omega: 21 routed, 33 dropped, for 0.39 success rate
4.0 omega: 41 routed, 13 dropped, for 0.76 success rate
9.472135899025261 omega: 41 routed, 13 dropped, for 0.76 success rate
10.0 omega: 41 routed, 13 dropped, for 0.76 success rate
12.0 omega: 41 routed, 13 dropped, for 0.76 success rate
